In [0]:
import os,sys
from ipyparallel import Client
from collections import defaultdict
import tempfile
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import socket
from multiprocessing import Pool, Manager
import shutil
import tempfile
from collections import OrderedDict
import pandas as pd
import numpy as np
import pickle
from Bio im

In [0]:
import shutil

In [0]:
from os import listdir as ls
from os import path as op

In [0]:
proc_files = !find /gpfs_fs/home/eckertlab/Mitra/SWWP_seq -name "*.fastq.gz" | grep -v trialRun | grep -v lane7

proc_files = sorted(proc_files[2:])

proc_files

In [0]:
#set up multiprocessing engines
#ipcluster start -n10 --profile=xmn
rc = Client(profile="sge")
dview = rc[:]
lview = rc.load_balanced_view()
len(lview)

In [0]:
def format_fastq_tuple(title, seq, qual):
    assert len(seq) == len(qual)
    return "@%s\n%s\n+\n%s\n" % (title, seq, qual)

def get_writers(barcodes, f):
    import tempfile
    from collections import defaultdict
    w = defaultdict()
    file_list = []
    for b, name in list(barcodes.items()):
        file_name = "%s_%s_%s.fastq" % (f, name, b)
        file_list.append(file_name)
        w[b] = open(file_name, "w")
    return w, file_list

def get_barcodes(f):
    from collections import defaultdict
    import os
    bcs = defaultdict()
    bc_lens = set()
    b = os.path.join(os.path.dirname(f), "barcodes.txt")
    h = open(b)
    h.readline()
    for line in h:
        line = line.strip()
        data = line.split("\t")
        if len(data) > 1: #skip blank lines
            if "," in data[0]:
                data[0] = data[0].replace(" ", "").replace(",", "-")
            bc = data[3].upper().replace("CTCTTTCCCTACACGACGCTCTTCCGATCT", "")[:-1]
            bc_lens.add(len(bc))
            bcs[bc] = data[1] + "_" + data[0]        
    return bcs, bc_lens    

def check_barcodes(barcodes):
    for i in list(barcodes.keys()):
        for j in list(barcodes.keys()):
            if i != j:
                assert not j.startswith(i)
                
def copy_file(src, dst):
    import shutil
    shutil.copy(src, dst)
                
def demult(f):
    print(f)
    import socket, os
    from Bio.SeqIO.QualityIO import FastqGeneralIterator
    from multiprocessing import Pool
    from collections import defaultdict
    import gzip
    hostname = socket.gethostname()
    out_dir = os.path.dirname(f)
    barcodes, barcode_lens = get_barcodes(f)
    check_barcodes(barcodes)
    writers, writer_list = get_writers(barcodes, f)
    count = 0
    found = defaultdict(int)
    bad_barcode = 0
    n_start = 0
    for title, seq, qual in FastqGeneralIterator(gzip.open(f, "rt")):

        if seq.startswith("N"):
            seq = seq[1:]
            qual = qual[1:]
            n_start += 1
        
        has_barcode = False    
        
        for l in barcode_lens: 
            bc = seq[0:l]
            if bc in barcodes:
                found[bc] += 1
                has_barcode = True
                w = writers[bc]
                w.write(format_fastq_tuple(title, seq[l:], qual[l:]))
                break
            
        count += 1
            
        if count % 100000 == 0:
            print(hostname, f, count)
        
        if not has_barcode:
            bad_barcode += 1
    
    for k, v in writers.items():
        v.close()
            
    return f, count, bad_barcode, found, writer_list

# dview['format_fastq_tuple'] = format_fastq_tuple
# dview['demult'] = demult
# dview['check_barcodes'] = check_barcodes
# dview['get_writers'] = get_writers
# dview['get_barcodes'] = get_barcodes
# dview['copy_file'] = copy_file


In [0]:
demult_jobs = []
for f in proc_files:
    demult_jobs.append(lview.apply_async(demult, f))

In [0]:
np.sum([j.ready() for j in demult_jobs])

In [0]:
[x.r for x in demult_jobs]

In [0]:
len(demult_jobs)

In [0]:
proc_files[0]

In [0]:
demult_results = [j.r for j in demult_jobs if j.ready()]

In [0]:
pickle.dump(demult_results, open("/gpfs_fs/home/eckertlab/Mitra/SWWP_seq/demult_results.pkl", "wb"))

In [0]:
demult_results = pickle.load(open("/gpfs_fs/home/eckertlab/Mitra/SWWP_seq/demult_results.pkl", "rb"))

In [0]:
demult_results

In [0]:
fastq_files = !find /gpfs_fs/home/eckertlab/Mitra/SWWP_seq | grep '.fastq$' | grep -v trial

In [0]:
unused = !find /gpfs_fs/home/eckertlab/Mitra/SWWP_seq | grep '.unused$' | grep -v trial

In [0]:
collapsed = [x for x in fastq_files if not "gz" in x]

In [0]:
collapsed

In [0]:
unused

In [0]:
for u in unused:
    fastq_files.append(u)

In [0]:
fastq_files[-10:]

In [0]:
samp_dict = {}
for f in fastq_files:
    d = f.split("_")
    sample = d[-2]
    if not sample in samp_dict:
        samp_dict[sample] = []
    samp_dict[sample].append(f)

In [0]:
for k, v in samp_dict.items():
    if len(v) > 1:
        print(k, v)

In [0]:
for k, v in samp_dict.items():
    if len(v) > 1:
        d = os.path.dirname(v[0])
        f = "_".join(v[0].split("_")[-2:])
        out = os.path.join(d, f).replace(".unused", "")
        print(out)
        with open(out, "w") as o:
            for elem in v:
                for name, seq, qual in FastqGeneralIterator(open(elem)):
                    o.write(format_fastq_tuple(name, seq, qual))

In [0]:
fastq_files = !find /gpfs_fs/home/eckertlab/Mitra/SWWP_seq | grep '.fastq$' | grep -v trial

In [0]:
len(fastq_files)

In [0]:
with open(os.path.join("/gpfs_fs/home/eckertlab/Mitra/SWWP_seq", "fastq_files.txt"), "w") as o:
    for f in fastq_files:
        o.write("%s\n" % f)